In [1]:
import scispacy
import spacy
from spacy import displacy
import pandas as pd
import random
from spacy.training.example import Example

In [2]:
sampleText="The most common adverse reactions across all TAXOTERE indications are infections, neutropenia, anemia, febrile neutropenia, hypersensitivity, thrombocytopenia, neuropathy, dysgeusia, dyspnea, constipation"

In [3]:
nlp = spacy.load("C:\ProgramData\Anaconda3\Lib\site-packages\en_ner_bc5cdr_md\en_ner_bc5cdr_md-0.4.0")

In [4]:
def AE(sampleText):
    doc = nlp(sampleText)
    #displacy_image = displacy.render(doc, jupyter = True, style = 'ent')
    entities=[]
    labels=[]
    for ent in doc.ents:
        entities.append(ent)
        labels.append(ent.label_)
    df= pd.DataFrame({'Entities':entities,'Labels':labels})
    return df

In [5]:
AEdetected=AE(sampleText)

In [6]:
AEdetected

,Entities,Labels
0,(infections),DISEASE
1,(neutropenia),DISEASE
2,(anemia),DISEASE
3,"(febrile, neutropenia)",DISEASE
4,(hypersensitivity),DISEASE
5,(thrombocytopenia),DISEASE
6,(neuropathy),DISEASE
7,(dysgeusia),DISEASE
8,(dyspnea),DISEASE
9,(constipation),DISEASE


nlp = spacy.load("C:\ProgramData\Anaconda3\Lib\site-packages\en_core_sci_sm\en_core_sci_sm-0.4.0")
#ner = nlp.add_pipe("ner")
ner.add_label("MY_LABEL")
nlp.to_disk(r"E:\testModel")

In [7]:
sampletext2="Traditional machine learning methods used to detect the side effects of drugs pose significant challenges as feature engineering processes are labor-intensive, expert-dependent, time-consuming and cost-ineffective. Moreover, these methods only focus on detecting the association between drugs and their side effects or classifying drug–drug interaction. Motivated by technological advancements and the availability of big data, we provide a review on the detection and classification of side effects using deep learning approaches. It is shown that the effective integration of heterogeneous, multidimensional drug data sources, together with the innovative deployment of deep learning approaches, helps reduce or prevent the occurrence of adverse drug reactions (ADRs). Deep learning approaches can also be exploited to find replacements for drugs which have side effects or help to diversify the utilization of drugs through drug repurposing."

In [10]:
#textData=pd.read_csv(r"C:\Users\Aadesh\Desktop\Aadesh\Birlasoft\Dataset\VAERS\2021VAERSDATA.csv",encoding = "ISO-8859-1")
#symptomsData=pd.read_csv(r"C:\Users\Aadesh\Desktop\Aadesh\Birlasoft\Dataset\VAERS\2021VAERSSYMPTOMS.csv",encoding = "ISO-8859-1")

In [11]:
#textData.shape

In [12]:
#symptomsData.shape

In [13]:
#textData.tail()

In [14]:
#MergedData=(pd.merge(textData, symptomsData, on='VAERS_ID'))

In [15]:
#MergedData.shape

In [16]:
#MergedData.head()

In [17]:
#cols = ['SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4',"SYMPTOM5"]
#MergedData['Symptoms'] = MergedData[cols].apply(lambda x: ','.join(x.dropna()), axis=1)

In [18]:
#MergedData['Symptoms'].head()

In [19]:
#trainingData= MergedData[['SYMPTOM_TEXT','Symptoms']]

In [20]:
#trainingData.head()

In [21]:
#trainingData.shape

In [22]:
#trainingData.to_csv('ScispacyTraining.csv')

In [2]:
trainingData = pd.read_csv(r"DataToBeUsed.csv")

In [3]:
data=list()
for index, row in trainingData.iterrows():
    temp=row["textCptured"]
    sym=temp.split(',')
    txt=str(row["SYMPTOM_TEXT"])
    flag2=0
    t=list()
    #print(txt)
    if ((txt)!=0):
        for i in range(len(sym)):
            #print(sym[i],txt)
            alert=0
            if(len(t)>0):
                for k in t:
                    if (txt.find(sym[i])) in k:
                        alert=1
                        break
                    else:
                        alert=0
            if alert ==0:
                if sym[i] in txt:
                    flag2=flag2+1
                    t.append((txt.find(sym[i]),(txt.find(sym[i])+len(sym[i])+1),"AE_Captured"))
    dic={'entities':t}
    if(len(t)!=0):
        tempData=(txt,dic)
    if(len(tempData)!=0):
        data.append(tempData)

In [4]:
len(data)

542

In [5]:
data

[('Adverse Events: Inflammation in the eye, confusion, headaches, inflammation in ears, cold chills, shivering, and fever like symptoms  Treatments: Primary care physician ran a series of bloodwork and found that after Flu shot I had big drop in white blood cell count and referred me to ophthalmologist and otolaryngologist  ophthalmologist  prescribed Cequa to treat the inflammation in eyes along with fortified caster oil.  otolaryngologist prescribed Prednisone to treat the inflammtion  Time course: Still having adverse events',
  {'entities': [(52, 61, 'AE_Captured')]}),
 ('Adverse Events: Inflammation in the eye, confusion, headaches, inflammation in ears, cold chills, shivering, and fever like symptoms  Treatments: Primary care physician ran a series of bloodwork and found that after Flu shot I had big drop in white blood cell count and referred me to ophthalmologist and otolaryngologist  ophthalmologist  prescribed Cequa to treat the inflammation in eyes along with fortified caste

In [7]:
def trainModel(trainData, iterations):
    TRAIN_DATA=trainData
    nlp = spacy.load("C:\ProgramData\Anaconda3\Lib\site-packages\en_ner_bc5cdr_md\en_ner_bc5cdr_md-0.4.0")
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe("ner")
    else:
        ner= nlp.get_pipe("ner")
    for _,annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        #optimizer=nlp.begin_training()
        for itn in range(iterations):
            print("starting iterartion"+str(itn))
            random.shuffle(TRAIN_DATA)
            losses={}
            for text,annotations in TRAIN_DATA:
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example],drop=0.2,losses=losses)
            print(losses)
    return (nlp)
            

In [8]:
NewTrainedModel = trainModel(data,60)

starting iterartion0


C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "I had a small burning sensation after the initial ..." with entities "[(171, 177, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "I had a series of symptoms that culminated with a ..." with entities "[(56, 62, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "injection site pain, tenderness, redness, swelling..." with entities "[(107, 116, 'AE_Captured'), (557, 564, 'AE_Capture...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "headache within minutes of receiving the vaccine. ..." with entities "[(0, 9, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\s

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "101 fever, severe chills, nausea and vomiting, spl..." with entities "[(4, 10, 'AE_Captured'), (56, 66, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "At 8pm on 123120 my left arm along with various ot..." with entities "[(108, 117, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\s

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Slight fever the day of the injections. The next n..." with entities "[(7, 13, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Patient received Moderna vaccine on 12/23/20 at ap..." with entities "[(234, 243, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.p

...

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Got vaccine at 11am,  was ok all day.  About 11pm ..." with entities "[(175, 181, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "On 12/24 patient experienced headache, fatigue, fe..." with entities "[(48, 54, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Pain at shot site/headache/joint pain/weak/stomach..." with entities "[(18, 27, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "employee c/o shakiness, rapid heart rate, headache..." with entities "[(42, 51, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Chest tightness, shivers, fever, headaches Fatigue" with entities "[(33, 42, 'AE_Captured'), (25, 32, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Soreness at injection site , high fever later that..." with entities "[(34, 40, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy

C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Fatigue, body aches and fever of 100.7" with entities "[(24, 30, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "patient had headache about 20 minutes after vaccin..." with entities "[(12, 21, 'AE_Captured')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\ProgramData\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:144: UserWarni

{'ner': 480.1018252110055}
starting iterartion1
{'ner': 294.1199020181954}
starting iterartion2
{'ner': 261.68218527214304}
starting iterartion3
{'ner': 243.14928642214176}
starting iterartion4
{'ner': 215.97117306999488}
starting iterartion5
{'ner': 186.99713412179383}
starting iterartion6
{'ner': 189.06334225323087}
starting iterartion7
{'ner': 182.97285792896494}
starting iterartion8
{'ner': 191.45075124503543}
starting iterartion9
{'ner': 183.86959503130234}
starting iterartion10
{'ner': 171.01648129870196}
starting iterartion11
{'ner': 147.4293870488893}
starting iterartion12
{'ner': 174.86043968326254}
starting iterartion13
{'ner': 151.27321637505295}
starting iterartion14
{'ner': 157.54616933012989}
starting iterartion15
{'ner': 149.45882517247696}
starting iterartion16
{'ner': 147.32217239421203}
starting iterartion17
{'ner': 149.08003969804525}
starting iterartion18
{'ner': 162.54412587633394}
starting iterartion19
{'ner': 135.6052913184001}
starting iterartion20
{'ner': 138.9

In [9]:
NewTrainedModel.to_disk(r"E:\Modelwith2AE60iter")

In [18]:
testData="Nasal congestion and diarrhea,12pm Received shot 1 pm Sore arm at injection site  8pm loss of appetite, whole arm hurting  11pm fever 101, severe chills, body ache,  tiredness 11:30pm-3am fever 103.6, chills, body ache, tiredness, headache, confusion, sore on right side of body, high blood pressure161/96, high heart rate 120?s while resting, slighty sob (O2 low 90?s) slight chest pain on right side under Rib front and back, nausea, tingling lips for about 5 mins maybe.  12-30-20  3am still had fever 101 after taking Tylenol, blood pressure and heart rate decreased a little, chills stopped,  was able to sleep  7am  fever was 102 nausea headache sore right arm   Headache and Fever 102 continued all day, fever was a little lower taking Tylenol 12-31-20  Had headache/pressure in head all day temp 98.6-100 all day, felt a lot better, right arm still a little sore"

In [19]:
#newnlp=spacy.load(r"E:\Modelwith2AE")

In [20]:
newdoc=NewTrainedModel(testData)

In [21]:
for ent in newdoc.ents:
    print(ent.text,ent.label_)

headache, AE_Captured
